# 第一节 单因素方差分析

## 实验5-1 单因素方差分析

检查三家工厂的机器完成生产所需平均时间是否相同。三家工厂机器完成生产所需时间如下

α=0.05

H0:三家工厂的机器完成生产所需平均时间相同
    
H1:三家工厂的机器完成生产所需平均时间不同

In [9]:
import pandas as pd 
import statsmodels.api as sm 
from statsmodels.formula.api import ols

In [10]:
productivity_df = pd.read_excel('5-1.xlsx')
productivity_df

,1,2,3
0,20,28,20
1,26,26,19
2,24,31,23
3,22,27,22


In [11]:
# 将原始数据的宽表转化成便于数据分析的长表
productivity_df_long = productivity_df.melt(var_name='factory' , value_name='time_spent')
productivity_df_long

,factory,time_spent
0,1,20
1,1,26
2,1,24
3,1,22
4,2,28
5,2,26
6,2,31
7,2,27
8,3,20
9,3,19


In [12]:
# 用pandas中groupby按工厂进行分组，然后用describe方法对各组进行描述性统计
productivity_df_long.groupby('factory').describe()

time_spent                                                
             count  mean       std   min    25%   50%    75%   max
factory                                                           
1              4.0  23.0  2.581989  20.0  21.50  23.0  24.50  26.0
2              4.0  28.0  2.160247  26.0  26.75  27.5  28.75  31.0
3              4.0  21.0  1.825742  19.0  19.75  21.0  22.25  23.0

In [18]:
productivity_lm = ols('time_spent~C(factory)' , data=productivity_df_long).fit()
sm.stats.anova_lm(productivity_lm)

,df,sum_sq,mean_sq,F,PR(>F)
C(factory),2.0,104.0,52.000000,10.636364,0.00426
Residual,9.0,44.0,4.888889,NaN,NaN


结论

由于p值 0.00426 < 0.05 ，因此在5%显著性水平下，应拒绝原假设，即三个厂机器完成工作时间是有显著差异的

## 第二节 双因素方差分析

## 实验5-2 无交互作用的双因素方差分析

三个品牌的手机在四个地区销售，销售数据如下。显著性水平5%，分析手机销售量是否由于品牌的不同和地区的不同而存在差异。

In [2]:
import pandas as pd 
import statsmodels.api as sm 
from statsmodels.formula.api import ols

In [3]:
sell_df = pd.read_excel('5-2.xlsx',index_col=0) # 注意索引
sell_df

,地区1,地区2,地区3,地区4
品牌1,4.5,6.4,7.2,6.7
品牌2,8.8,7.8,9.6,7.0
品牌3,5.9,6.8,5.7,5.2


In [4]:
# 清洗数据，变长表
sell_df_n = sell_df.reset_index()
sell_df_n = sell_df_n.rename(index = str , columns = {'index':'brand'})
sell_df_n

,brand,地区1,地区2,地区3,地区4
0,品牌1,4.5,6.4,7.2,6.7
1,品牌2,8.8,7.8,9.6,7.0
2,品牌3,5.9,6.8,5.7,5.2


In [5]:
sell_df_long = sell_df_n.melt(id_vars='brand' , var_name='area' , value_name='sell')
sell_df_long

,brand,area,sell
0,品牌1,地区1,4.5
1,品牌2,地区1,8.8
2,品牌3,地区1,5.9
3,品牌1,地区2,6.4
4,品牌2,地区2,7.8
5,品牌3,地区2,6.8
6,品牌1,地区3,7.2
7,品牌2,地区3,9.6
8,品牌3,地区3,5.7
9,品牌1,地区4,6.7


In [6]:
sell_lm = ols('sell~C(brand)+C(area)' , data=sell_df_long).fit()
sm.stats.anova_lm(sell_lm) 

,df,sum_sq,mean_sq,F,PR(>F)
C(brand),2.0,13.68,6.840000,6.237082,0.034258
C(area),3.0,2.82,0.940000,0.857143,0.512185
Residual,6.0,6.58,1.096667,NaN,NaN


结论

对于品牌因素 p值0.034258 < 0.05 ， 拒绝原假设，即品牌是重要的影响因素。

对于地区因素 p值0.512185 > 0.05 ， 拒绝无法原假设，即品牌不是重要的影响因素。

## 实验5-3 有交互作用的双因素方差分析

五一与十一两个黄金周，四个旅游线路，旅游情况如下，显著性水平5%

判断：

旅游线路之间是否存在差异？

两个黄金周之间是否存在差异？

是否存在线路与黄金周交互作用的影响？

In [7]:
import pandas as pd 
import statsmodels.api as sm 
from statsmodels.formula.api import ols

In [8]:
tourist_df = pd.read_excel('5-3.xlsx' , index_col=0) # 注意索引
tourist_df

,线路A,线路B,线路C,线路D
五一,31,22,14,8
NaN,29,23,16,4
NaN,32,26,20,6
NaN,30,25,15,5
NaN,30,24,18,5
十一,25,21,16,5
NaN,22,20,13,7
NaN,27,16,15,8
NaN,26,19,12,7
NaN,22,15,10,8


In [9]:
# 数据清洗，转换表
tourist_df_n = tourist_df.reset_index()

# 重命名黄金周变量为period，原变量名为index
tourist_df_n = tourist_df_n.rename(index=str , columns = {'index':'period'})

# 填充原始数据中黄金周变量的缺失值
tourist_df_n.loc[0:5 , 'period'] = '五一'
tourist_df_n.loc[6:10 , 'period'] = '十一'

tourist_df_n

,period,线路A,线路B,线路C,线路D
0,五一,31,22,14,8
1,五一,29,23,16,4
2,五一,32,26,20,6
3,五一,30,25,15,5
4,五一,30,24,18,5
5,十一,25,21,16,5
6,十一,22,20,13,7
7,十一,27,16,15,8
8,十一,26,19,12,7
9,十一,22,15,10,8


In [10]:
# 转换长表
tourist_df_long = tourist_df_n.melt(id_vars='period' , var_name='plan' , value_name='amount')
tourist_df_long

,period,plan,amount
0,五一,线路A,31
1,五一,线路A,29
2,五一,线路A,32
3,五一,线路A,30
4,五一,线路A,30
5,十一,线路A,25
6,十一,线路A,22
7,十一,线路A,27
8,十一,线路A,26
9,十一,线路A,22


In [11]:
tourist_lm = ols('amount~C(period)+C(plan)+C(period):C(plan)' , data=tourist_df_long).fit()
sm.stats.anova_lm(tourist_lm) 

,df,sum_sq,mean_sq,F,PR(>F)
C(period),1.0,119.025,119.025000,30.617363,4.208408e-06
C(plan),3.0,2431.475,810.491667,208.486602,4.475032e-21
C(period):C(plan),3.0,88.875,29.625000,7.620579,5.564285e-04
Residual,32.0,124.400,3.887500,NaN,NaN


结论

对于period因素，4.208408e-06 < 0.05 ， 拒绝原假设，即不同时期有显著影响

对于plan因素，4.475032e-21 < 0.05 ， 拒绝原假设，即不同旅游路线有显著影响

对于交互因素，5.564285e-04 < 0.05 ， 拒绝原假设，旅游线路与黄金周的交互作用对游客人数产生显著影响（旅游景点具有一定季节性特征，两者相互作用影响游客人数）